# Worksheet 08

Name:  Shengduo Li
UID: U64101332

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [49]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.540214828606472, 5.211883248850736, 4.761447666892855, 4.788075242880893, 5.826212880651104, 2.8855017782410877, 6.046235186072743, 5.4977917463977395, 6.441537333496354, 4.993132616359467]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [50]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[8.177220542317578, 7.518794736452263, 7.612131794301775, 7.868401414177628, 9.4926705733176, 9.149732649809613, 8.007606874313337, 8.819412774050402, 7.45004811184517, 8.067447709549665]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [51]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.067447709549665, 4.993132616359467, 7.45004811184517, 6.441537333496354, 8.819412774050402, 8.007606874313337, 5.4977917463977395, 9.149732649809613, 9.4926705733176, 7.868401414177628]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean and variance for the two normal distributions, and the number of data that follow each of the two normal distributions.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [52]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) ,len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2)  ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.067447709549665, 7.45004811184517, 8.819412774050402, 8.007606874313337, 9.149732649809613, 9.4926705733176, 7.868401414177628]
[4.993132616359467, 6.441537333496354, 5.4977917463977395]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 8.407902872437631,  mean_2 = 5.6441538987511874
var_1 = 0.4830095646427481,  var_2 = 0.3603569772581648


/Users/alexisli/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


They are not very close from the true value.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [53]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0]*prob_s[0])/prob_x )
    prob_s1_x.append((pdf_i[1]*prob_s[1])/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.067447709549665
probability of observing that point if it came from cluster 0 =  0.6442715118863129
probability of observing that point if it came from cluster 1 =  1.676577039363564e-10
point =  4.993132616359467
probability of observing that point if it came from cluster 0 =  1.157587207441127e-11
probability of observing that point if it came from cluster 1 =  0.2164962572652796
point =  7.45004811184517
probability of observing that point if it came from cluster 0 =  0.11560728245233828
probability of observing that point if it came from cluster 1 =  3.896776588644277e-06
point =  6.441537333496354
probability of observing that point if it came from cluster 0 =  0.00020798940483416842
probability of observing that point if it came from cluster 1 =  0.09571010695641517
point =  8.819412774050402
probability of observing that point if it came from cluster 0 =  0.5745623956272405
probability of observing that point if it came from cluster 1 =  1.5296718132299816e-17
point =

The cluster I think each data comes from based on the estimation are the same as the truth.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [54]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7005030601530765,  P(S_2) = 0.29949693984692344
mean_1 = 8.406488659832352,  mean_2 = 5.642819421930599
var_1 = 0.48544355703485875,  var_2 = 0.3599051824849819



The new estimates of mean_j,var_j, and P(S_j) are all slightly different with the these values from the original ones.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [55]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0]*prob_c[0])/prob_x )
    prob_s1_x.append((pdf_i[1]*prob_c[1])/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.067447709549665
probability of observing that point if it came from cluster 0 =  0.6439476317470305
probability of observing that point if it came from cluster 1 =  1.5468696017547265e-10
point =  4.993132616359467
probability of observing that point if it came from cluster 0 =  1.509498916939073e-11
probability of observing that point if it came from cluster 1 =  0.2173324610481645
point =  7.45004811184517
probability of observing that point if it came from cluster 0 =  0.11798827108613368
probability of observing that point if it came from cluster 1 =  3.710796173039782e-06
point =  6.441537333496354
probability of observing that point if it came from cluster 0 =  0.00022749945889827784
probability of observing that point if it came from cluster 1 =  0.09446282712825899
point =  8.819412774050402
probability of observing that point if it came from cluster 0 =  0.5723430876585167
probability of observing that point if it came from cluster 1 =  1.3445547095350986e-17
point 

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [56]:
assignment=[]
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    if p[1]>=p[2]:
        assignment.append([p[0],1])
    else:
        assignment.append([p[0],0])

for i in assignment:
    print("point: "+str(i[0])+" belongs to cluster "+str(i[1]))
   

point: 8.067447709549665 belongs to cluster 1
point: 4.993132616359467 belongs to cluster 0
point: 7.45004811184517 belongs to cluster 1
point: 6.441537333496354 belongs to cluster 0
point: 8.819412774050402 belongs to cluster 1
point: 8.007606874313337 belongs to cluster 1
point: 5.4977917463977395 belongs to cluster 0
point: 9.149732649809613 belongs to cluster 1
point: 9.4926705733176 belongs to cluster 1
point: 7.868401414177628 belongs to cluster 1
